Here im doing transfer learning and fine tunning with a resnet50 and pytorch, to classify bees and ants

In [1]:
#@title colab sincronization and dict of labels
from google.colab import drive
drive.mount('/content/drive')

label_dic = {0:'Ant',
             1:'Bee'}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
from PIL import Image
import copy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, dataset
from torchvision import transforms, datasets, models
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# First we have to find the means and stds of our train dataset to lately Normalize

# Define a transform to convert images to tensors
to_tensor = transforms.ToTensor()

# Initialize lists to store the tensors and the total number of tensors
tensors = []
total_tensors = 0
directories = ['/content/drive/MyDrive/Colab Notebooks/resnet/data_resnet50/train/bees',
               '/content/drive/MyDrive/Colab Notebooks/resnet/data_resnet50/train/ants']
# Iterate over the directories
for directory in directories:
    # Load all the images in the directory
    images = []
    for file in os.listdir(directory):
        if file.endswith('.jpg'):
            image = Image.open(os.path.join(directory, file))
            images.append(image)

    # Convert the images to tensors and add them to the list
    tensors.extend([to_tensor(image) for image in images])
    total_tensors += len(images)

# Calculate the mean and std of the tensors for each channel
means_data = []
stds_data = []
for i in range(3):  # 3 channels (R, G, B)
    mean = 0.
    std = 0.
    for tensor in tensors:
        mean += tensor[i, :, :].mean()
        std += tensor[i, :, :].std()
    mean /= total_tensors
    std /= total_tensors
    means_data.append(mean)
    stds_data.append(std)



In [4]:
# Define a transform to preprocess the input images
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(means_data, stds_data) # Normalize the pixel values
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(means_data, stds_data) # Normalize the pixel values
    ])
}

# Import Data

data_dir = '/content/drive/MyDrive/Colab Notebooks/resnet/data_resnet50'
sets = ['train', 'validation']
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir, x), 
                                         data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size = 4,
                             shuffle = True)
               for x in ['train', 'validation']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes
print(class_names)

# Training 
def train_model(model, criterion, optimazer, scheduler, num_epochs = 10):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        #Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train() # set the model in training mode
            else:
                model.eval()  # set the model in evaluation mode
            running_loss = 0.0
            running_corrects = 0
                
                
            # iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #forward
                #track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                #backward + optimize, only if is training phase
                if phase == 'train':
                    # Clean the gradients of the parameters to optimize. 
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step() # Updating the weights
            
            #statistics
            running_loss +=loss.item()* inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        if phase == 'train':
            scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double()/ dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        #deep copy the model 
        if phase == 'validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    print()
    
    print(f'Training Complete, Best Val Acc: {best_acc:.4f}')
    
    # Load best model Weights
    model.load_state_dict(best_model_wts)
    return model

['ants', 'bees']


In [5]:
model = models.resnet50(pretrained = True)

# Now we will freeze the first layers of the NN, to train the weights only 
# of the latest layer
for param in model.parameters():
    param.requires_grad = False

# The resnet50 was trained on ImageNet dataset, that's why it has 1000 labels in 
# the output layer (out_features), in our implementation, we want to classify only 
# 2 labels(bees & ants, so, we will edit the last layer of the resnet.

num_classes = 2
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)


# model to GPU
device = torch.device("cuda:0") 
model.to(device)


# Define a loss function and an optimizer
criterion = torch.nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.Adadelta(model.parameters(), lr = learning_rate)

# Every 7 epochs our learning rate its only update 10%.
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma = 0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs= 11)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 1/10
----------
validation Loss: 0.0304 Acc: 0.0000
Epoch 2/10
----------
validation Loss: 0.0189 Acc: 0.0000
Epoch 3/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 4/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 5/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 6/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 7/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 8/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 9/10
----------
validation Loss: 0.0000 Acc: 0.0067
Epoch 10/10
----------
validation Loss: 0.0000 Acc: 0.0067

Training Complete, Best Val Acc: 0.0067


OPTIONAL: SAVING THE WEIGHTS

In [6]:
# torch.save(model.state_dict(), "./resnet50_finetuned.pth") # saving the weights

In [7]:
# model_loaded = models.resnet50(pretrained = True) # Loading the weights that i saved before

# num_classes = 2
# num_ftrs = model_loaded.fc.in_features
# model_loaded.fc = nn.Linear(num_ftrs, num_classes)

# model_loaded.load_state_dict(torch.load("./resnet50_finetuned.pth"))
# model_loaded.to(device)

In [10]:
# To resize and crop the image, you can use the Resize and CenterCrop transforms
# To convert the image to a tensor and normalize its pixel values, you can use 
# the ToTensor and Normalize 

from PIL import Image

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(means_data, stds_data)
])


X_test = Image.open('/content/drive/MyDrive/Colab Notebooks/resnet/test_resnet50/hormiga1.jpg')
image_tensor = transform_test(X_test)
image_tensor = image_tensor.unsqueeze(0)

image_tensor = image_tensor.to(device)

y_pred = model(image_tensor)
_, pred = torch.max(y_pred, 1)
label_dic[pred.item()]

'Ant'

In [11]:
X_test = Image.open('/content/drive/MyDrive/Colab Notebooks/resnet/test_resnet50/hormiga2.jpg')
image_tensor = transform_test(X_test)
image_tensor = image_tensor.unsqueeze(0)

image_tensor = image_tensor.to(device)

y_pred = model(image_tensor)
_, pred = torch.max(y_pred, 1)
label_dic[pred.item()]

'Ant'

In [12]:
X_test = Image.open('/content/drive/MyDrive/Colab Notebooks/resnet/test_resnet50/aveja1.jpg')
image_tensor = transform_test(X_test)
image_tensor = image_tensor.unsqueeze(0)

image_tensor = image_tensor.to(device)

y_pred = model(image_tensor)
_, pred = torch.max(y_pred, 1)
label_dic[pred.item()]

'Bee'

In [13]:
X_test = Image.open('/content/drive/MyDrive/Colab Notebooks/resnet/test_resnet50/aveja2.jpg')
image_tensor = transform_test(X_test)
image_tensor = image_tensor.unsqueeze(0)

image_tensor = image_tensor.to(device)

y_pred = model(image_tensor)
_, pred = torch.max(y_pred, 1)
label_dic[pred.item()]

'Bee'

In [14]:
X_test = Image.open('/content/drive/MyDrive/Colab Notebooks/resnet/test_resnet50/amarillo_negro.jpg')
image_tensor = transform_test(X_test)
image_tensor = image_tensor.unsqueeze(0)

image_tensor = image_tensor.to(device)

y_pred = model(image_tensor)
_, pred = torch.max(y_pred, 1)
label_dic[pred.item()]

'Bee'